In [ ]:
# # Calculate the propagator for T
# # assumes Ampltiude * envelope(t) is real/constant

# # v1 integrate Pulse H1(t) = [c e^{-iwt} - c^* e^{iwt}] from 0 to T
# U_t = (-1j * (H0 * T + H1 * (2 * c * 1j * (np.cos(wp * T) - 1) / wp))).expm()

# v2 integate the Pulse H1(t) = c * cos(wp * t) from 0 to T
# U_t = (-1j * (H0 * T + H1 * (c * np.sin(wp * T) / wp))).expm()
# # Apply the propagator to the initial state
# rho_t = qt.Qobj(U_t * rho0 * U_t.dag())

# # Calculate fidelities
# best_fidelity = qt.fidelity(rho_t, expected_rho)
# avg_gate_fidelity = qt.average_gate_fidelity(desired_U, U_t)
# print(f"Best fidelity: {best_fidelity:.4f} with c = {c:.4f}")
# print(f"Average gate fidelity: {avg_gate_fidelity:.4f}")

#####################################
# try using qutip's propagator
# turns out to be the same as our manual integral calculation
period_time = np.linspace(0, T, 250)  # a single period of the pulse
pulse = Pulse(omega=wp, amp=30)
H_pump = qs.hamiltonian.driven_term(snail_mode=snail)  # n_max=3.0)
H = [qs.hamiltonian.H0, [H_pump, pulse.drive]]
U_ts = qt.propagator(H, period_time, c_ops, args=args, options=opts)

# plot occupations for each time step inside the period
period_occupations = {mode: np.zeros(len(period_time)) for mode in qs.modes_num}
for idx, t in enumerate(period_time):
    rho_tt = qt.Qobj(U_ts[idx] * rho0 * U_ts[idx].dag())
    for mode in qs.modes_num:
        period_occupations[mode][idx] = np.abs(qt.expect(qs.modes_num[mode], rho_tt))

# next, propagate the state 1 period at a time
U_t = U_ts[-1]  # propagator for T=1 period
n_periods = 250
full_time = np.linspace(0, n_periods * T, n_periods)  # 250 periods of the pulse
occupations = {mode: np.zeros(len(full_time)) for mode in qs.modes_num}

rho_tt = rho0
for idx, t in tqdm(enumerate(full_time)):
    rho_tt = qt.Qobj(U_t * rho_tt * U_t.dag())
    for mode in qs.modes_num:
        # cast to reals, but we expect the imaginary part to be zero
        occupations[mode][idx] = np.abs(qt.expect(qs.modes_num[mode], rho_tt))

In [ ]:
import numpy as np
import qutip as qt
from tqdm import tqdm

# Define the pulse frequency and amplitude
detuning = 0.0535  # -0.1 * 2 * np.pi
pulse_frequency = np.abs(qubit1.freq - qubit2.freq) + detuning
T = 2 * np.pi / pulse_frequency
amplitude = 14.15

# Define pulse on and off configurations
pulse_on = Pulse(omega=pulse_frequency, amp=amplitude)
pulse_off = Pulse(omega=pulse_frequency, amp=0)

# Define Hamiltonians for pulse on and off
H_pump = qs.hamiltonian.driven_term(snail_mode=snail)  # n_max=None)
H_on = [qs.hamiltonian.H0, [H_pump, pulse_on.drive]]
H_off = [qs.hamiltonian.H0, [H_pump, pulse_off.drive]]

# Calculate propagators for on and off periods
period_time = np.linspace(0, T, 250)  # Duration of one pulse period
U_on = qt.propagator(H_on, period_time, c_ops, args=args, options=opts)[-1]
U_off = qt.propagator(H_off, period_time, c_ops, args=args, options=opts)[-1]

n_off_periods = 20
n_on_periods = 300
n_periods = 2 * n_off_periods + n_on_periods
full_time = np.linspace(
    0, n_periods * T, n_periods
)  # Time array for the whole simulation


# Initial state
rho_tt = rho0
# Occupation dictionaries for both dressed and undressed bases
occupations = {mode: np.zeros(n_periods) for mode in qs.modes_num}
# und_occupations = {mode: np.zeros(n_periods) for mode in qs.modes_num}

# Simulate evolution: first off, then on, then off
for idx in tqdm(range(n_periods)):
    if idx < n_off_periods or idx >= n_off_periods + n_on_periods:
        U_t = U_off  # Pulse off
    else:
        U_t = U_on  # Pulse on

    rho_tt = qt.Qobj(U_t * rho_tt * U_t.dag())

    # Calculate occupations in the dressed basis
    for mode in qs.modes_num:
        occupations[mode][idx] = np.abs(qt.expect(qs.modes_num[mode], rho_tt))

# Plotting the occupations
import matplotlib.pyplot as plt

# Assuming qs.modes_num and other variables are already defined and calculated

# Setup the figure and axes for subplots
fig, axs = plt.subplots(2, 1, figsize=(6, 6))

# Plotting fine-grained evolution within one period
for mode, occ in period_occupations.items():
    axs[0].plot(period_time, occ, label=mode.name, marker="*")
axs[0].set_title("Mode Occupations Over 1 Period")
axs[0].set_xlabel("Time")
axs[0].set_ylabel("Occupation Number")
axs[0].legend()
axs[0].grid(True)

# Plotting full time evolution over n_periods
for mode, occ in occupations.items():
    alpha = 0.5 if mode is snail else 1.0
    axs[1].plot(full_time, occ, label=mode.name, marker="o", alpha=alpha)
axs[1].set_title(f"Mode Occupations Over {n_on_periods} Periods")
axs[1].set_xlabel("Time")
axs[1].set_ylabel("Occupation Number")
axs[1].legend()
# put vertical lines to indicate on and off periods
axs[1].axvline(n_off_periods * T, color="black", linestyle="--")
axs[1].axvline((n_off_periods + n_on_periods) * T, color="black", linestyle="--")
axs[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import qutip as qt
from tqdm import tqdm
import matplotlib.pyplot as plt


def propagate(
    pulse_frequency,
    amplitude,
    snail,
    qs,
    rho0,
    c_ops,
    args,
    opts,
):
    T = 2 * np.pi / pulse_frequency  # Calculate period of the pulse
    target_gate_time = 500
    # Calculate the number of on periods to approximate the target gate time
    n_on_periods = int(np.floor(target_gate_time / T))

    # Define pulse on and off configurations
    pulse_on = Pulse(omega=pulse_frequency, amp=amplitude)
    pulse_off = Pulse(omega=pulse_frequency, amp=0)

    # Define Hamiltonians for pulse on and off
    H_pump = qs.hamiltonian.driven_term(snail_mode=snail, n_max=None)
    H_on = [qs.hamiltonian.H0, [H_pump, pulse_on.drive]]
    H_off = [qs.hamiltonian.H0, [H_pump, pulse_off.drive]]

    # Propagators for on and off periods
    period_time = np.linspace(0, T, 250)  # Duration of one pulse period
    U_on = qt.propagator(H_on, period_time, c_ops, args=args, options=opts)[-1]
    U_off = qt.propagator(H_off, period_time, c_ops, args=args, options=opts)[-1]

    # Total periods including off periods at start and end
    n_off_periods = 10
    n_periods = 2 * n_off_periods + n_on_periods
    # Time array for the whole simulation
    full_time = np.linspace(0, n_periods * T, n_periods)
    # occupations = {mode: np.zeros(n_periods) for mode in qs.modes_num}
    rho_tt = rho0

    # Simulate the pulse off, on, off sequence
    for idx in range(n_periods):
        if idx < n_off_periods or idx >= n_off_periods + n_on_periods:
            U_t = U_off  # Pulse off
        else:
            U_t = U_on  # Pulse on

        rho_tt = qt.Qobj(U_t * rho_tt * U_t.dag())
        # for mode in qs.modes_num:
        #     occupations[mode][idx] = np.abs(qt.expect(qs.modes_num[mode], rho_tt))

    return _, _, rho_tt


def parameter_sweep(
    detuning_range,
    amplitude_range,
    snail,
    qs,
    rho0,
    c_ops,
    args,
    opts,
    wp,
):
    fidelity_matrix = np.zeros((len(amplitude_range), len(detuning_range)))

    for i, amp in tqdm(enumerate(amplitude_range)):
        for j, detuning in enumerate(detuning_range):
            pulse_frequency = wp + 2 * np.pi * (
                detuning / 1000
            )  # Convert detuning from MHz to rad/s
            _, _, final_state = propagate(
                pulse_frequency, amp, snail, qs, rho0, c_ops, args, opts
            )
            qubit_rhof = final_state.ptrace(range(len(qubits)))
            fidelity_matrix[i, j] = qt.fidelity(qubit_rhof, expected_qubit_rho)

    plt.figure(figsize=(8, 6))
    plt.imshow(
        fidelity_matrix,
        extent=(
            detuning_range[0],  # Start of detuning range in MHz
            detuning_range[-1],  # End of detuning range in MHz
            amplitude_range[0],  # Start of amplitude range
            amplitude_range[-1],  # End of amplitude range
        ),
        aspect="auto",
        origin="lower",
        cmap="viridis",
    )
    plt.colorbar(label="Fidelity")
    plt.xlabel("Detuning (MHz)")
    plt.ylabel("Amplitude")
    plt.title("Fidelity across Frequency and Amplitude")
    plt.show()


# Define parameter ranges for the sweep
N = 25
detuning_range = np.linspace(-10, 10, N)  # Detuning in MHz
amplitude_range = np.linspace(0, 30, N)  # Example amplitude range

# Execute the parameter sweep
parameter_sweep(detuning_range, amplitude_range, snail, qs, rho0, c_ops, args, opts, wp)

In [ ]:
def propagate_for_optimization(
    pulse_frequency, amplitude, snail, qs, rho0, c_ops, args, opts
):
    T = 2 * np.pi / pulse_frequency
    period_time = np.linspace(0, T, 250)
    n_periods = int(np.floor(150 / T))
    pulse = Pulse(omega=pulse_frequency, amp=amplitude)
    H_pump = qs.hamiltonian.driven_term(snail_mode=snail, n_max=None)
    H = [qs.hamiltonian.H0, [H_pump, pulse.drive]]
    U_ts = qt.propagator(H, period_time, c_ops, args=args, options=opts)
    U_t = U_ts[-1]
    full_time = np.linspace(0, n_periods * T, n_periods)
    # occupations = {mode: np.zeros(len(full_time)) for mode in qs.modes_num}
    rho_tt = rho0
    for idx, t in enumerate(full_time):
        rho_tt = qt.Qobj(U_t * rho_tt * U_t.dag())
        # for mode in qs.modes_num:
        #     occupations[mode][idx] = np.abs(qt.expect(qs.modes_num[mode], rho_tt))
    return rho_tt


def objective_function(
    x, snail, qs, rho0, c_ops, args, opts, qubits, expected_qubit_rho
):
    pulse_frequency, amplitude = x
    final_state = propagate_for_optimization(
        pulse_frequency, amplitude, snail, qs, rho0, c_ops, args, opts
    )
    fidelity = extract_fidelity(final_state, qubits, expected_qubit_rho)
    return 1 - fidelity


def callback(intermediate_result):
    x = intermediate_result.x
    current_fidelity = 1 - intermediate_result.fun
    print(
        f"Current params: pulse_frequency={x[0]}, amplitude={x[1]} - Current Fidelity: {current_fidelity}"
    )


# Example usage in optimization:
ret = minimize(
    fun=objective_function,
    x0=[wp + 0.0535, 14.15],
    args=(snail, qs, rho0, c_ops, args, opts, qubits, expected_qubit_rho),
    # bounds=[(wp - 0.2, wp + 0.2), (0, 20)],
    callback=callback,
    method="Nelder-Mead",
)